In [ ]:
from google.colab import drive
drive.mount ('/content/drive')


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
data = pd.read_csv('/content/drive/My Drive/Dataset/congressional_voting_records/house-votes-84.data', index_col = None , header = None , na_values = '?')


In [ ]:
data.columns =['Class', 'handicapped-infants',
'water-project-cost-sharing' , 'adoption-of-the-budget-resolution',
'physician-fee-freeze' , 'el-salvador-aid',
'religious-groups-in-schools','anti-satellite-test-ban' ,
'aid-to-nicaraguan-contras' ,'mx-missile' , 'immigration' ,
'synfuels-corporation-cutback','education-spending',
'superfund-right-to-sue' , 'crime' ,'duty-free-exports','export-administration-act-south-africa' ]
print(data.head())

In [ ]:
print(data.info())

In [ ]:
print(data.shape)

In [6]:
#import pandas as pd
data = data.drop_duplicates(keep = 'first')

In [ ]:
print(data.shape)

In [ ]:
print(data.isnull().sum().sort_values(ascending = False))

In [ ]:
clsmap = {'republican' : 1, 'democrat':0}
data['republican'] = data['Class'].map(clsmap)
data.drop('Class' , axis = 1 , inplace = True)
for column in data.columns.drop('republican'):
  data[column+'1'] = data[column].map({'y' : 1 , 'n' : 0, '?' :np.nan })
  data.drop(column, axis = 1,  inplace = True)
print(data.head())


In [ ]:
#Make a copy of original data
df1 = data.copy(deep = True)
#df1.replace('?' , np.nan , inplace = True)
df1.replace ({'n' : 0 , 'y' : 1} , inplace = True)
print(df1.head())
print(df1.dtypes)
print(df1.shape)

In [ ]:
print(df1.head())

In [12]:
#Mode Imputation
factor_cols = df1.columns[1 :]
for col in factor_cols:
  df1.fillna(df1[col].mode()[0] , inplace = True)

In [ ]:
print(df1.isnull().sum())

In [ ]:
#Should done after splitting
import seaborn as sns
df_matrix = df1.corr()
sns.heatmap(df_matrix, annot = True)
plt.show()

In [ ]:
X = df1.iloc[: ,1 : 17]
y = df1.iloc[: ,0]
print(X.shape , y.shape)

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest , chi2
X_train , X_test , y_train , y_test = train_test_split(X , y , train_size = 0.8, random_state = 42 , shuffle = True)

selector = SelectKBest(score_func = chi2,k = 12)
X_train_selected = selector.fit_transform(X_train , y_train)
X_test_selected = selector.transform(X_test)

In [ ]:
#Names of choosen attributes
names = X_train.columns.values[selector.get_support()]
print(names)

In [ ]:
print(X_train_selected.shape)
print(X_test_selected.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#Check if dataset is Imbalanced
import numpy as np
lab , counts = np.unique(y_test , return_counts = True)
p = counts/len(y_test)
print(p)
print(y_test)

In [20]:
#Building decision tree model
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score

In [21]:
#Build Decision Tree model
dt_cls = DecisionTreeClassifier(criterion= 'gini', min_samples_split = 2)
dt_cls = dt_cls.fit(X_train_selected , y_train)
dt_y_pred = dt_cls.predict(X_test_selected)
from sklearn.tree import plot_tree
#tree.plot_tree(dt_cls)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
tn , fp , fn , tp = confusion_matrix(y_test, dt_y_pred).ravel()
print("Model Accuracy: %.3f"% accuracy_score(y_test , dt_y_pred))

In [23]:
print(tp , tn , fp , fn)

23 42 2 2


In [ ]:
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f"%accuracy)
print("Precision: %.3f"%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f "%specificity)
print("Sensitivity: %.3f"%sensitivity)


In [ ]:
#Counting the number of each class
import numpy as np
unique , counts = np.unique(y_test , return_counts = True)
print(dict(zip(unique , counts)))

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
tree.plot_tree(dt_cls , filled =True)
plt.rcParams['figure.figsize']

In [27]:
from sklearn.tree import _tree

def tree_to_code(tree , feature_names):
  tree_ = tree.tree_
  feature_name = [
      feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
      for i in tree_.feature
  ]
  print('def tree({}): '.format(" , ".join(feature_names)))
  def recurse(node , depth):
    indent = " " * depth
    if tree_.feature[node] != _tree.TREE_UNDEFINED:
      name = feature_name[node]
      threshold = tree_.threshold[node]
      print("{} if <= {} : ".format(indent , name ,  threshold))
      recurse(tree_.children_left[node] , depth+1)
      print("{} else: # if {} > {}".format(indent , name , threshold))
      recurse(tree_.children_right[node], depth+1)
    else:
      print("{} return {}".format(indent,  tree_.value[node]))
  recurse(0 , 1)

In [65]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
  tree_ = tree.tree_
  feature_name = [
      feature_names[i] if i != _tree.TREE_UNDEFINED and i < len(feature_names) else "undefined!"  # Added a condition to check index bounds
      for i in tree_.feature
  ]
  print('def tree({}): '.format(" , ".join(feature_names)))

  def recurse(node, depth):
    indent = " " * depth
    if tree_.feature[node] != _tree.TREE_UNDEFINED:
      name = feature_name[node]
      threshold = tree_.threshold[node]
      print("{} if <= {} : ".format(indent, name, threshold))
      recurse(tree_.children_left[node], depth + 1)
      print("{} else: # if {} > {}".format(indent, name, threshold))
      recurse(tree_.children_right[node], depth + 1)
    else:
      print("{} return {}".format(indent, tree_.value[node]))

  recurse(0, 1)

In [ ]:
tree_to_code(dt_cls ,names)

In [ ]:
#Build optimized Decision Tree model
dt_cls = DecisionTreeClassifier(criterion = 'entropy' , max_depth = 4, min_samples_split= 2)
dt_cls = dt_cls.fit(X_train , y_train)

y_pred = dt_cls.predict(X_test)
print("Optimized Decision Tree Accuracy: %.3f"% metrics.accuracy_score(y_test , y_pred))
tree.plot_tree(dt_cls)
plt.figure()
from sklearn import tree
import matplotlib.pyplot as plt
tree.plot_tree(dt_cls , filled =True)
plt.rcParams['figure.figsize']

In [ ]:

tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
print(tn , fp , fn , tp)
accuracy = (tp+tn) /(tn+fp+fn+tp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f"% accuracy)
print("Precision: %.3f"%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f "%specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
tree_to_code(dt_cls ,names)

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
tree.plot_tree(dt_cls , filled =True)
plt.rcParams['figure.figsize']

In [ ]:
#Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression
logistic_Reg = LogisticRegression(penalty = 'l2' ,C = 10 , random_state = 0, solver = 'liblinear').fit(X_train_selected , y_train)
y_pred = logistic_Reg.predict(X_test_selected)
print("Logistic Regression Accuracy: %.3f"%metrics.accuracy_score(y_test , y_pred))


In [ ]:
lr_Cmat = confusion_matrix(y_test , y_pred)
print(lr_Cmat)
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp+tn)/(tp+tn + fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f"%accuracy)
print("Precision: %.3f"%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f"%specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
from sklearn.model_selection import GridSearchCV
param = {'penalty': ['l1' , 'l2'] ,
          'C': [0.0001 ,0.001 , 0.01 , 1 , 10 , 100]}
model = LogisticRegression()
grid_search = GridSearchCV(model, param, cv = 5)
grid_search.fit(X_train_selected , y_train)
print(grid_search.best_params_)
print(grid_search.best_score_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_selected)
print('accuracy: %.3f'%accuracy_score(y_test , y_pred))


In [ ]:
lr_Cmat = confusion_matrix(y_test , y_pred)
print(lr_Cmat)
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = tp/(tp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f"%accuracy)
print("Precision: %.3f"%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f"%specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
#Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression
logistic_Reg = LogisticRegression( C = 0.01, solver = 'liblinear').fit(X_train_selected , y_train)
y_pred = logistic_Reg.predict(X_test_selected)
print("Logistic Regression Accuracy: %.3f"% metrics.accuracy_score(y_test , y_pred))

In [ ]:
lr_Cmat = confusion_matrix(y_test , y_pred)
print(lr_Cmat)
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp+tn)/(tn+ fp+tp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy:%.3f"% accuracy)
print("Precision: %.3f " %precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f "%specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
from sklearn import tree
import matplotlib.pyplot as plt
tree.plot_tree(dt_cls , filled =True)
plt.rcParams['figure.figsize']

In [ ]:

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier( criterion = 'gini' , random_state = 0 , min_samples_split = 3)
rfc.fit(X_train_selected , y_train)
y_pred = rfc.predict(X_test_selected)
print("Model Accuracy: %.3f"%accuracy_score(y_test , y_pred) )

In [ ]:
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
print(tp , tn , fp , fn)
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f"%accuracy)
print("Precision: %.3f"%precision)
print("Recall: .%.3f"%recall)
print("Specificity: %.3f "% specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
rfc = RandomForestClassifier(criterion = 'entropy' ,max_depth = 4, random_state =1, min_samples_split = 2)
rfc.fit(X_train_selected , y_train)
y_pred = rfc.predict(X_test_selected)
print("Model Accuracy: %.3f"%accuracy_score(y_test , y_pred) )

In [ ]:

rfc_Cmat = confusion_matrix(y_test , y_pred)
print(rfc_Cmat)
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f"% accuracy)
print("Precision: %.3f"%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f"%specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
#Building Neural Network
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(random_state = 1 , max_iter = 50).fit(X_train_selected , y_train)
nn.predict(X_test_selected )

In [ ]:
nn_Cmat = confusion_matrix(y_test , y_pred)
print(rfc_Cmat)
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: " , accuracy)
print("Precision: " ,precision)
print("Recall: ", recall)
print("Specificity: ", specificity)
print("Sensitivity: ",sensitivity)

In [ ]:
#Building Neural Network
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes = (5,),activation = "relu", solver = 'adam', alpha = 0.001,learning_rate = 'constant' ,random_state = 1 , max_iter = 200 ).fit(X_train_selected , y_train)
y_pred = nn.predict(X_test_selected )
print("Model Accuracy: %.3f"%accuracy_score(y_test , y_pred) )

In [ ]:
nn_Cmat = confusion_matrix(y_test , y_pred)
print(nn_Cmat)
tn , fp , fn , tp = confusion_matrix(y_test, y_pred).ravel()
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f "% accuracy)
print("Precision: %.3f "%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f"%specificity)
print("Sensitivity: %.3f"%sensitivity)


In [ ]:
#Building Neural Network
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes = (3,),activation = "relu", solver = 'lbfgs', alpha = 0.0001,learning_rate = 'constant' ,random_state = 1 , max_iter = 300 ).fit(X_train_selected , y_train)
y_pred = nn.predict(X_test_selected )
print("Model Accuracy: %.3f"%accuracy_score(y_test , y_pred) )

In [ ]:
tn , fn , fp ,tp = confusion_matrix(y_test, y_pred).ravel()
print(tn , fn , fp , tp)
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: %.3f "% accuracy)
print("Precision: %.3f "%precision)
print("Recall: %.3f"%recall)
print("Specificity: %.3f"%specificity)
print("Sensitivity: %.3f"%sensitivity)

In [ ]:
import tensorflow as tf
print("tensorflow version: ", tf.__version__)

In [ ]:
model = Sequential()
model.add(Dense(units = 5, activation = 'relu' , input_shape = (12,)))
#model.add(Dense(units = 2, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid')) # Change to 1 unit and sigmoid activation
adam = keras.optimizers.Adam(learning_rate = 0.001)

In [ ]:
model.compile(loss = 'binary_crossentropy' , optimizer = adam , metrics = ['accuracy'])
model.fit(X_train_selected, y_train, epochs = 100)
loss_metrics = model.evaluate(X_test_selected , y_test)
print(model.summary())
print('loss: %.3f'%loss_metrics[0])
print('Accuracy: %.3f'%loss_metrics[1])

y_predict = model.predict(X_test_selected)

In [ ]:
from sklearn.metrics import confusion_matrix , ConfusionMatrixDisplay
y_predict = tf.squeeze(y_predict)
y_predict = np.array([1 if x ==1 else 0 for x in y_predict])
actual = np.array(y_test)
confusion_mat = confusion_matrix(actual , y_predict)
displ = ConfusionMatrixDisplay(confusion_matrix = confusion_mat)
displ.plot()

In [ ]:

result = confusion_matrix(y_test, np.rint (y_predict))
print(result)

In [ ]:
#tn , fp , fn , tp =unzip(confusion_mat)
tn , fp , fn  , tp = result.ravel()
accuracy = (tp+tn)/(tp+tn+fp+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: " , accuracy)
print("Precision: " ,precision)
print("Recall: ", recall)
print("Specificity: ", specificity)
print("Sensitivity: ",sensitivity)
recall = tp / (tp + fn)
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
print("Accuracy: " , accuracy)
print("Precision: " ,precision)
print("Recall: ", recall)
print("Specificity: ", specificity)
print("Sensitivity: ",sensitivity)
